## 품질계획 현황 수집

In [ ]:
from common import commonFunc as cf
import pandas as pd
import threading

pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "품질계획 현황"
APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

print("실행 금지")
exit()

preSetFolder

## 연도별 공사계약 목록에서  SPTNO 활용

In [ ]:
#임시 
imsiDf = pd.read_csv(\
         "../output/건설사업정보시스템/연도별 공사계약 목록/selectIoCmProjConstYearContractList.csv", encoding="ms949")

In [ ]:
dfCombi = imsiDf.loc[:,["sptno"]].drop_duplicates()
dfCombi.reset_index(inplace=True, drop=True)

In [ ]:
dfCombiLen = dfCombi.shape[0]
dfCombiLen

In [ ]:
numOfThread = 10

resultDfMerged = list(pd.DataFrame() for x in range(numOfThread))


In [ ]:
##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
SERVICENAME

In [ ]:
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]
PRIMARYKEY

In [ ]:
JSONKEY = "detail1"
DUMMY = 0

In [ ]:
#### 페이지번호 설정 ###
PAGEYN=1
if REQPARAM.count("pageyn") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1
PAGEYN

## 보고년월 기간을 추출하기 위한 커스텀 함수

In [ ]:
from datetime import datetime

# d1, d2 : datetime format 
# d1 >= d2
def calc_month(d1, d2):
    return (d1.year - d2.year) * 12 + (d1.month - d2.month + 1)

## 품질계획 현황 전용 스크랩 함수

In [ ]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os

STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn, jsonkey="items", dummy=0, inType="jsonabnormal"):
    try:
        emptyPd = pd.DataFrame()
        i=1
        while True:
            print("{} page scraping start".format(i))
            
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

            response = requests.get(inUrl+queryParams, verify=False)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata != "NORMAL_SERVICE"):
                        print("{} page is empty".format(i))
                        break
                    
#               여기서부터는 데이터는 있는데, totalcount>0 경우는 detailList1~5까지 봐야 한다.
               
                jsondata = response.json()["response"]["body"][jsonkey]
                if( jsondata == []):
                    print("{} page is empty".format(i))
                    break
                if( jsonkey == "detail1"):
                    jsondata["index"]=[0]
                rowData = pd.DataFrame(jsondata)
                emptyPd = emptyPd.append(rowData)
                print(">totalCount "+str(response.json()["response"]["body"]["totalCount"]))
                if(0 < int(response.json()["response"]["body"]["totalCount"])):
                    if(response.json()["response"]["body"].get('detailList1') != None):
                        print("we have a data on detailList1")
                        jsondataDetail = response.json()["response"]["body"]["detailList1"]

                    elif(response.json()["response"]["body"].get('detailList2') != None):
                        jsondataDetail = response.json()["response"]["body"]["detailList2"]
                        print("we have a data on detailList2")

                    elif(response.json()["response"]["body"].get('detailList3') != None):
                        jsondataDetail = response.json()["response"]["body"]["detailList3"]
                        print("we have a data on detailList3")

                    elif(response.json()["response"]["body"].get('detailList4') != None):
                        jsondataDetail = response.json()["response"]["body"]["detailList4"]
                        print("we have a data on detailList4")

                    elif(response.json()["response"]["body"].get('detailList5') != None):
                        jsondataDetail = response.json()["response"]["body"]["detailList5"]
                        print("we have a data on detailList5")
#                     print(jsondataDetail)
                    rowDataDetail = pd.DataFrame(jsondataDetail)
                    emptyPd = pd.merge(emptyPd, rowDataDetail, on='sptNo')
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return emptyPd       
    except Exception as e:
            print(e)     

## 테스트용 코드

In [ ]:
'''
resultDfMerged = pd.DataFrame()
resultDf = pd.DataFrame()

BASEPARAM={"serviceKey":APIKEY,"sptNo": sptnoValue, "rprtYm": "", "type":"json"}
resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
if resultDf.empty: # 정상 데이터가 없는 경우
    print("No Data")
else:
    tmpStartDay = resultDf["stwrdt"].values[0].split('-')
    tmpEndDay = resultDf["ccwdt"].values[0].split('-')

    dateRange = pd.period_range(resultDf["stwrdt"].values[0],resultDf["ccwdt"].values[0],freq='M').strftime('%Y%m').tolist()
    print(dateRange)
    for i in range(0,len(dateRange)):
        BASEPARAM={"serviceKey":APIKEY,"sptNo": sptnoValue, "rprtYm": str(dateRange[i]), "type":"json"}
        resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
        resultDfMerged = resultDfMerged.append(resultDf) 
        print("----------------------------------------")
        print(resultDf)
'''

In [ ]:
lock = threading.Lock()
def threadReq(startIdx, endIdx,x):
    resultDf = pd.DataFrame()
    global resultDfMerged

    for i in range(startIdx,endIdx):
        sptnoValue = dfCombi.loc[i].sptno
        BASEPARAM={"serviceKey":APIKEY,"sptNo": sptnoValue, "rprtYm": "", "type":"json"}
        resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
        print("i= ",i)
        if resultDf.empty: # 정상 데이터가 없는 경우
            print("No Data")
        else:
            dateRange = pd.period_range(resultDf["stwrdt"].values[0],resultDf["ccwdt"].values[0],freq='M').strftime('%Y%m').tolist()
            for i in range(0,len(dateRange)):
                BASEPARAM={"serviceKey":APIKEY,"sptNo": sptnoValue, "rprtYm": str(dateRange[i]), "type":"json"}
                resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
                lock.acquire()
                resultDfMerged[x] = resultDfMerged[x].append(resultDf) 
                lock.release()


In [ ]:

resultDfLastMerged = pd.DataFrame()
section = list(range(0, dfCombiLen, int(dfCombiLen/numOfThread))) # we have "numOfThread +1" sections

# dfCombiLen = 6 # for test
if dfCombiLen < numOfThread:
    print("데이터의 갯수가 적어서 쓰레드를 시작할 수 없습니다.")
else:
    x_ls =list(range(numOfThread))
    thread_list = []
    results = []

    for x in x_ls:
        startIdx = section[x]
        endIdx =  section[x+1] -1 if (x+1 <= numOfThread -1) else dfCombiLen - 1
        print("start ",startIdx, " end ",endIdx)
        mythread = threading.Thread(target=threadReq, args=(startIdx, endIdx+1,x))
        thread_list.append(mythread)
    for i in thread_list:
        i.start()
    for i in thread_list:
        i.join()
    for i in range(0,numOfThread):
        resultDfLastMerged = resultDfLastMerged.append(resultDfMerged[i]) 



In [ ]:
# for i in range(0,numOfThread):
#     resultDfLastMerged = resultDfLastMerged.append(resultDfMerged[i]) 

In [ ]:
# [TEST] 
# resultDfMerged = list(pd.DataFrame())
# data = [1,2,3,4,5]
# mypd = pd.DataFrame(data)
# resultDfMerged.append(mypd)
# resultDfMerged.append(mypd)
# resultDfMerged.append(mypd)
# resultDfMerged[2]

In [ ]:
# 쓰레드 사용하지 않는 기존 코드
# for i in range(0,dfCombiLen):
#     sptnoValue = dfCombi.loc[i].sptno
#     BASEPARAM={"serviceKey":APIKEY,"sptNo": sptnoValue, "rprtYm": "", "type":"json"}
#     resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
#     print("i= ",i)
#     if resultDf.empty: # 정상 데이터가 없는 경우
#         print("No Data")
#     else:
# #         tmpStartDay = resultDf["stwrdt"].values[0].split('-')
# #         tmpEndDay = resultDf["ccwdt"].values[0].split('-')

#         dateRange = pd.period_range(resultDf["stwrdt"].values[0],resultDf["ccwdt"].values[0],freq='M').strftime('%Y%m').tolist()
#         print(dateRange)
#         for i in range(0,len(dateRange)):
#             BASEPARAM={"serviceKey":APIKEY,"sptNo": sptnoValue, "rprtYm": str(dateRange[i]), "type":"json"}
#             resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
#             resultDfMerged = resultDfMerged.append(resultDf) 
# #             print("----------------------------------------")
# #             print(resultDf)
        

In [ ]:
resultDfLastMerged.shape

In [ ]:
cf.savedata(resultDfLastMerged, SITENAME,DATANAME,SERVICENAME)

In [ ]:
resultDfLastMerged.shape